Reference MNIST code: https://github.com/pytorch/examples/blob/master/mnist/main.py

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt

In [2]:
input_size = 784 # img_size = (28,28) ---> 28*28=784 in total
hidden_size = 500 # number of nodes at hidden layer
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 1 # number of times which the entire dataset is passed throughout the model
batch_size = 64  # the size of input data took for one iteration
lr = 5e-3 # size of step
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [3]:
train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

In [4]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

In [5]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
  
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [6]:
net = Net(input_size, hidden_size, num_classes).to(device)

In [7]:
optimizer = optim.Adam(net.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()

In [8]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device).view(-1,28*28), target.to(device)
        optimizer.zero_grad()
        output = model(data)
#         loss = F.nll_loss(output, target)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

In [9]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device).view(-1,28*28), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [10]:
for epoch in range(1, num_epochs + 1):
    train(net, device, train_gen, optimizer, epoch)
    test(net, device, test_gen)
test(net, device, train_gen)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.292077
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.293656
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.258743
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.328236
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.162212
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.174384
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.077574
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.089459
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.062161
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.139637

Test set: Average loss: -8.9732, Accuracy: 9554/10000 (96%)


Test set: Average loss: -8.9527, Accuracy: 57874/60000 (96%)



96.45666666666666

In [11]:
weights = dict(net.named_parameters())['fc1.weight'].detach().cpu().numpy()
weights.shape

(500, 784)